In [37]:
from pathlib import Path
import pandas as pd
import json
import os


## Evaluation of different allocation approaches

In [38]:
directory = Path("testsets5/testset5/evaluation")

In [39]:
# Get list of all considered files
files = []
for filepath in directory.rglob("*"):  # Matches all files and directories
    extension = os.path.splitext(filepath)[-1]
    if filepath.is_file() and extension not in [ ".log", ".png"]:
        files.append(filepath)
files

[PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-skill_short_branch-3-uniform.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-random-3-early.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-skill_short_branch-3-early.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-random-3-uniform.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-random-3-uniform.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-skill_short_branch-3-early.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-random-3-early.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-random-3-early.xml.json'),
 PosixPath('testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-random-3-uniform.xml.json'),
 PosixPath('testsets5/testset5/e

In [40]:
data = {}
for file in files:
    print(file)
    with open(file) as f:
        object = json.load(f)
        object 
    try:
        data[str(file)] = object["solution"]
    except:
        KeyError
        print(f"excluded file: {file}")
        


testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-skill_short_branch-3-uniform.xml.json
testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-random-3-early.xml.json
testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-skill_short_branch-3-early.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-random-3-uniform.xml.json
testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-random-3-uniform.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-skill_short_branch-3-early.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-random-3-early.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-random-3-early.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.6, 0.4, 0.0)-random-3-uniform.xml.json
testsets5/testset5/evaluation/single_instance_cp/(1, 0, 0.0)-skill_short_branch-3-uniform.xml.json
testsets5/testset5/evaluation/single_instance_cp/(0.8, 0.2, 0.0)-ski

In [41]:
df = pd.DataFrame.from_dict(data, orient="index")
df = df.reset_index().rename(columns={"index":'solution_set'})
df["solution_approach"] = df["solution_set"].str.split("/").apply(lambda x: x[3] if len(x) > 3 else None)
df["metadata"] = df["solution_set"].str.split("/").apply(lambda x: x[-1] if len(x) > 4 else None)

df['flex'] = df['metadata'].str.split("_").apply(lambda x: x[0])  # First element
df['cp'] = df['metadata'].str.split("_").apply(lambda x: x[1] if len(x) > 1 else None)  # Second element
df['d_type'] = df['metadata'].str.split("_").apply(lambda x: x[2:] if len(x) > 2 else [])  # Rest of the elements
df['d_type'] = df['d_type'].apply(lambda x: "".join(x))

In [42]:
df

,solution_set,objective,optimality gap,lower_bound,computing time,solver status,branches,propagations,total interval length,solution_approach,metadata,flex,cp,d_type
0,testsets5/testset5/evaluation/single_instance_...,77.0,N/A,77.0,43.30,Optimal,1358136.0,N/A,241.0,single_instance_cp,"(0.6, 0.4, 0.0)-skill_short_branch-3-uniform.x...","(0.6, 0.4, 0.0)-skill",short,branch-3-uniform.xml.json
1,testsets5/testset5/evaluation/single_instance_...,126.0,N/A,126.0,9.28,Optimal,359112.0,N/A,440.0,single_instance_cp,"(1, 0, 0.0)-random-3-early.xml.json","(1, 0, 0.0)-random-3-early.xml.json",None,
2,testsets5/testset5/evaluation/single_instance_...,152.0,N/A,152.0,4.66,Optimal,727478.0,N/A,395.0,single_instance_cp,"(1, 0, 0.0)-skill_short_branch-3-early.xml.json","(1, 0, 0.0)-skill",short,branch-3-early.xml.json
3,testsets5/testset5/evaluation/single_instance_...,76.0,N/A,76.0,38.91,Optimal,1323753.0,N/A,245.0,single_instance_cp,"(0.8, 0.2, 0.0)-random-3-uniform.xml.json","(0.8, 0.2, 0.0)-random-3-uniform.xml.json",None,
4,testsets5/testset5/evaluation/single_instance_...,102.0,N/A,102.0,33.43,Optimal,1310659.0,N/A,352.0,single_instance_cp,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)-random-3-uniform.xml.json",None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,testsets5/testset5/evaluation/single_instance_...,65.0,N/A,65.0,12.25,Optimal,1134837.0,N/A,286.0,single_instance_cp_shift,"(0.8, 0.2, 0.0)-random-3-early.xml.json","(0.8, 0.2, 0.0)-random-3-early.xml.json",None,
68,testsets5/testset5/evaluation/single_instance_...,51.0,N/A,51.0,203.31,Optimal,2291912.0,N/A,258.0,single_instance_cp_shift,"(0.6, 0.4, 0.0)-random-3-uniform.xml.json","(0.6, 0.4, 0.0)-random-3-uniform.xml.json",None,
69,testsets5/testset5/evaluation/single_instance_...,117.0,N/A,117.0,304.22,Optimal,11161239.0,N/A,340.0,single_instance_cp_shift,"(1, 0, 0.0)-skill_short_branch-3-uniform.xml.json","(1, 0, 0.0)-skill",short,branch-3-uniform.xml.json
70,testsets5/testset5/evaluation/single_instance_...,74.0,N/A,74.0,54.33,Optimal,1207411.0,N/A,305.0,single_instance_cp_shift,"(0.8, 0.2, 0.0)-skill_short_branch-3-uniform.x...","(0.8, 0.2, 0.0)-skill",short,branch-3-uniform.xml.json


In [43]:

df["set_ident"] =  df.groupby(['flex', 'cp', 'd_type']).ngroup() + 1
df = df.sort_values(by=["d_type", "flex", "cp", "objective" ])
fin_df = df[["d_type", "flex", "cp", "objective", "solution_approach", "set_ident" ]]
fin_df

,d_type,flex,cp,objective,solution_approach,set_ident
18,,"(0.6, 0.4, 0.0)-random-3-early.xml.json",None,66.0,single_instance_cp_replan,NaN
42,,"(0.6, 0.4, 0.0)-random-3-early.xml.json",None,66.0,all_instance_cp_new,NaN
66,,"(0.6, 0.4, 0.0)-random-3-early.xml.json",None,66.0,single_instance_cp_shift,NaN
54,,"(0.6, 0.4, 0.0)-random-3-early.xml.json",None,79.0,heuristic_new,NaN
6,,"(0.6, 0.4, 0.0)-random-3-early.xml.json",None,81.0,single_instance_cp,NaN
...,...,...,...,...,...,...
21,branch-3-uniform.xml.json,"(1, 0, 0.0)-skill",short,97.0,single_instance_cp_replan,6.0
57,branch-3-uniform.xml.json,"(1, 0, 0.0)-skill",short,105.0,heuristic_new,6.0
69,branch-3-uniform.xml.json,"(1, 0, 0.0)-skill",short,117.0,single_instance_cp_shift,6.0
9,branch-3-uniform.xml.json,"(1, 0, 0.0)-skill",short,120.0,single_instance_cp,6.0


In [44]:
# Count the occurrences of each set_ident
set_ident_counts = df['set_ident'].value_counts()

# Filter for identifiers that appear 4 or more times
valid_identifiers = set_ident_counts[set_ident_counts >= 4].index

# Return rows where the set_ident is in valid_identifiers
filtered_df = df[df['set_ident'].isin(valid_identifiers)][["d_type", "flex", "cp", "objective", "solution_approach", "set_ident" ]]
filtered_df

,d_type,flex,cp,objective,solution_approach,set_ident
71,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,70.0,single_instance_cp_shift,1.0
47,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,72.0,all_instance_cp_new,1.0
23,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,75.0,single_instance_cp_replan,1.0
59,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,87.0,heuristic_new,1.0
11,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,88.0,single_instance_cp,1.0
35,branch-3-early.xml.json,"(0.6, 0.4, 0.0)-skill",short,113.0,single_instance_heuristic,1.0
41,branch-3-early.xml.json,"(0.8, 0.2, 0.0)-skill",short,86.0,all_instance_cp_new,3.0
17,branch-3-early.xml.json,"(0.8, 0.2, 0.0)-skill",short,87.0,single_instance_cp_replan,3.0
65,branch-3-early.xml.json,"(0.8, 0.2, 0.0)-skill",short,89.0,single_instance_cp_shift,3.0
53,branch-3-early.xml.json,"(0.8, 0.2, 0.0)-skill",short,103.0,heuristic_new,3.0


In [45]:
df = pd.DataFrame.from_dict(data, orient="index")
df = df.reset_index().rename(columns={"index":'solution_set'})
df["solution_approach"] = df["solution_set"].str.split("/").apply(lambda x: x[3] if len(x) > 3 else None)
df["metadata"] = df["solution_set"].str.split("/").apply(lambda x: x[-1] if len(x) > 4 else None)

df['flex'] = df['metadata'].str.split("-").apply(lambda x: x[0])  # First element
df['cp'] = df['metadata'].str.split("-").apply(lambda x: x[1] if len(x) > 1 else None)  # Second element
df['d_type'] = df['metadata'].str.split("-").apply(lambda x: x[2:] if len(x) > 2 else [])  # Rest of the elements
df['branch_nesting'] = df['metadata'].str.split("-").apply(lambda x: x[3:] if len(x) > 3 else [])  # Rest of the elements
df['flex_distribution'] = df['metadata'].str.split("-").apply(lambda x: x[4:] if len(x) > 4 else [])  # Rest of the elements
df['d_type'] = df['d_type'].apply(lambda x: "".join(x))

In [46]:
df

,solution_set,objective,optimality gap,lower_bound,computing time,solver status,branches,propagations,total interval length,solution_approach,metadata,flex,cp,d_type,branch_nesting,flex_distribution
0,testsets5/testset5/evaluation/single_instance_...,77.0,N/A,77.0,43.30,Optimal,1358136.0,N/A,241.0,single_instance_cp,"(0.6, 0.4, 0.0)-skill_short_branch-3-uniform.x...","(0.6, 0.4, 0.0)",skill_short_branch,3uniform.xml.json,[uniform.xml.json],[]
1,testsets5/testset5/evaluation/single_instance_...,126.0,N/A,126.0,9.28,Optimal,359112.0,N/A,440.0,single_instance_cp,"(1, 0, 0.0)-random-3-early.xml.json","(1, 0, 0.0)",random,3early.xml.json,[early.xml.json],[]
2,testsets5/testset5/evaluation/single_instance_...,152.0,N/A,152.0,4.66,Optimal,727478.0,N/A,395.0,single_instance_cp,"(1, 0, 0.0)-skill_short_branch-3-early.xml.json","(1, 0, 0.0)",skill_short_branch,3early.xml.json,[early.xml.json],[]
3,testsets5/testset5/evaluation/single_instance_...,76.0,N/A,76.0,38.91,Optimal,1323753.0,N/A,245.0,single_instance_cp,"(0.8, 0.2, 0.0)-random-3-uniform.xml.json","(0.8, 0.2, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[]
4,testsets5/testset5/evaluation/single_instance_...,102.0,N/A,102.0,33.43,Optimal,1310659.0,N/A,352.0,single_instance_cp,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,testsets5/testset5/evaluation/single_instance_...,65.0,N/A,65.0,12.25,Optimal,1134837.0,N/A,286.0,single_instance_cp_shift,"(0.8, 0.2, 0.0)-random-3-early.xml.json","(0.8, 0.2, 0.0)",random,3early.xml.json,[early.xml.json],[]
68,testsets5/testset5/evaluation/single_instance_...,51.0,N/A,51.0,203.31,Optimal,2291912.0,N/A,258.0,single_instance_cp_shift,"(0.6, 0.4, 0.0)-random-3-uniform.xml.json","(0.6, 0.4, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[]
69,testsets5/testset5/evaluation/single_instance_...,117.0,N/A,117.0,304.22,Optimal,11161239.0,N/A,340.0,single_instance_cp_shift,"(1, 0, 0.0)-skill_short_branch-3-uniform.xml.json","(1, 0, 0.0)",skill_short_branch,3uniform.xml.json,[uniform.xml.json],[]
70,testsets5/testset5/evaluation/single_instance_...,74.0,N/A,74.0,54.33,Optimal,1207411.0,N/A,305.0,single_instance_cp_shift,"(0.8, 0.2, 0.0)-skill_short_branch-3-uniform.x...","(0.8, 0.2, 0.0)",skill_short_branch,3uniform.xml.json,[uniform.xml.json],[]


In [47]:

df["set_ident"] =  df.groupby(['flex', 'cp', 'd_type']).ngroup() + 1
df = df.sort_values(by=["d_type", "flex", "cp", "objective" ])
fin_df = df[["d_type", "flex", "cp", "objective", "solution_approach", "set_ident" ]]
fin_df

,d_type,flex,cp,objective,solution_approach,set_ident
18,3early.xml.json,"(0.6, 0.4, 0.0)",random,66.0,single_instance_cp_replan,1
42,3early.xml.json,"(0.6, 0.4, 0.0)",random,66.0,all_instance_cp_new,1
66,3early.xml.json,"(0.6, 0.4, 0.0)",random,66.0,single_instance_cp_shift,1
54,3early.xml.json,"(0.6, 0.4, 0.0)",random,79.0,heuristic_new,1
6,3early.xml.json,"(0.6, 0.4, 0.0)",random,81.0,single_instance_cp,1
...,...,...,...,...,...,...
21,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,97.0,single_instance_cp_replan,12
57,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,105.0,heuristic_new,12
69,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,117.0,single_instance_cp_shift,12
9,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,120.0,single_instance_cp,12


In [48]:
# Count the occurrences of each set_ident
set_ident_counts = df['set_ident'].value_counts()

# Filter for identifiers that appear 4 or more times
valid_identifiers = set_ident_counts[set_ident_counts >= 4].index

# Return rows where the set_ident is in valid_identifiers
filtered_df = df[df['set_ident'].isin(valid_identifiers)][["d_type", "flex", "cp", "branch_nesting", "flex_distribution", "objective", "solution_approach", "set_ident" ]]
filtered_df

,d_type,flex,cp,branch_nesting,flex_distribution,objective,solution_approach,set_ident
18,3early.xml.json,"(0.6, 0.4, 0.0)",random,[early.xml.json],[],66.0,single_instance_cp_replan,1
42,3early.xml.json,"(0.6, 0.4, 0.0)",random,[early.xml.json],[],66.0,all_instance_cp_new,1
66,3early.xml.json,"(0.6, 0.4, 0.0)",random,[early.xml.json],[],66.0,single_instance_cp_shift,1
54,3early.xml.json,"(0.6, 0.4, 0.0)",random,[early.xml.json],[],79.0,heuristic_new,1
6,3early.xml.json,"(0.6, 0.4, 0.0)",random,[early.xml.json],[],81.0,single_instance_cp,1
...,...,...,...,...,...,...,...,...
21,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,[uniform.xml.json],[],97.0,single_instance_cp_replan,12
57,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,[uniform.xml.json],[],105.0,heuristic_new,12
69,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,[uniform.xml.json],[],117.0,single_instance_cp_shift,12
9,3uniform.xml.json,"(1, 0, 0.0)",skill_short_branch,[uniform.xml.json],[],120.0,single_instance_cp,12


In [51]:
# Filter rows for "single_instance_pc" and "heuristic"
single_instance_pc = df[df["solution_approach"] == "single_instance_cp_shift"]
heuristic = df[df["solution_approach"] == "heuristic_new"]

# Merge the filtered rows on "set_ident" to allow comparison
merged = pd.merge(
    single_instance_pc[["set_ident", "objective"]],
    heuristic[["set_ident", "objective"]], 
    on="set_ident",
    suffixes=("_pc", "_heuristic")
)

# Find set_idents where the "objective_pc" < "objective_heuristic"
valid_set_idents = merged[merged["objective_pc"] < merged["objective_heuristic"]]["set_ident"]

# Select all rows from the original DataFrame with these set_idents
result = df[df["set_ident"].isin(valid_set_idents)]

# Print the result
result

,solution_set,objective,optimality gap,lower_bound,computing time,solver status,branches,propagations,total interval length,solution_approach,metadata,flex,cp,d_type,branch_nesting,flex_distribution,set_ident
18,testsets5/testset5/evaluation/single_instance_...,66.0,N/A,27.0,253.050000,Feasible,2191837.0,N/A,282.0,single_instance_cp_replan,"(0.6, 0.4, 0.0)-random-3-early.xml.json","(0.6, 0.4, 0.0)",random,3early.xml.json,[early.xml.json],[],1
42,testsets5/testset5/evaluation/all_instance_cp_...,66.0,N/A,27.0,200.170000,Feasible,8155488.0,N/A,282.0,all_instance_cp_new,"(0.6, 0.4, 0.0)-random-3-early.xml.json","(0.6, 0.4, 0.0)",random,3early.xml.json,[early.xml.json],[],1
66,testsets5/testset5/evaluation/single_instance_...,66.0,N/A,66.0,11.940000,Optimal,943375.0,N/A,268.0,single_instance_cp_shift,"(0.6, 0.4, 0.0)-random-3-early.xml.json","(0.6, 0.4, 0.0)",random,3early.xml.json,[early.xml.json],[],1
54,testsets5/testset5/evaluation/heuristic_new/(0...,79.0,NaN,NaN,2.937560,NaN,NaN,NaN,296.0,heuristic_new,"(0.6, 0.4, 0.0)-random-3-early.xml.json","(0.6, 0.4, 0.0)",random,3early.xml.json,[early.xml.json],[],1
6,testsets5/testset5/evaluation/single_instance_...,81.0,N/A,81.0,6.790000,Optimal,350111.0,N/A,282.0,single_instance_cp,"(0.6, 0.4, 0.0)-random-3-early.xml.json","(0.6, 0.4, 0.0)",random,3early.xml.json,[early.xml.json],[],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,testsets5/testset5/evaluation/all_instance_cp_...,79.0,N/A,55.0,200.150000,Feasible,4845724.0,N/A,356.0,all_instance_cp_new,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[],10
16,testsets5/testset5/evaluation/single_instance_...,83.0,N/A,55.0,205.490000,Feasible,1350584.0,N/A,353.0,single_instance_cp_replan,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[],10
52,testsets5/testset5/evaluation/heuristic_new/(1...,91.0,NaN,NaN,6.099404,NaN,NaN,NaN,331.0,heuristic_new,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[],10
4,testsets5/testset5/evaluation/single_instance_...,102.0,N/A,102.0,33.430000,Optimal,1310659.0,N/A,352.0,single_instance_cp,"(1, 0, 0.0)-random-3-uniform.xml.json","(1, 0, 0.0)",random,3uniform.xml.json,[uniform.xml.json],[],10


In [50]:
merged

,set_ident,objective_pc,objective_heuristic
0,1,81.0,79.0
1,3,88.0,87.0
2,5,87.0,77.0
3,7,106.0,103.0
4,9,126.0,121.0
5,11,152.0,129.0
6,2,71.0,75.0
7,4,77.0,70.0
8,6,76.0,68.0
9,8,85.0,96.0
